# Global-Conflict-Analysis

# Imports

In [1]:
import pandas as pd

# Functions

# EDA Analysis

In [3]:
df = pd.read_csv("/Users/maukanmir/Downloads/data.csv")
df

,fromSocial,text,likesCount,commentsCount,viewsCount,input,authorMeta/name,creationDate
0,youtube,NaN,NaN,NaN,NaN,yemencivilwar,NaN,NaN
1,youtube,Russia-Ukraine Conflict: Putin Warns NATO Risk...,22.0,7.0,3069.0,russiaukraineconflict,CNN-News18,2024-09-13T16:23:39.000Z
2,youtube,LIVE: Russia Launches Waves of Drone Attacks o...,118.0,21.0,16358.0,russiaukraineconflict,Firstpost,2024-10-01T00:17:49.000Z
3,youtube,Russian Forces Take Over Ukraine's Avdiivka | ...,166.0,56.0,18054.0,russiaukraineconflict,CNBC-TV18,2024-02-19T15:37:34.000Z
4,youtube,Russia-Ukraine War: Ukraine's Surprise Attack ...,53.0,4.0,6096.0,russiaukraineconflict,DD India,2024-08-24T13:36:51.000Z
...,...,...,...,...,...,...,...,...
2541,youtube,Russia Ukraine War: उत्तर पश्चिम यूक्रेन पर बड...,2600.0,55.0,217668.0,russiaukraineconflict,News Nation,2024-08-20T13:35:00.000Z
2542,youtube,Russia Claims US Using Ukraine As Proxy. #russ...,323.0,11.0,5355.0,russiaukraineconflict,THE CHANAKYA DIALOGUES ENGLISH,2024-04-23T10:00:22.000Z
2543,youtube,Russia Ukraine Conflict live updates |Russian ...,352.0,1.0,51781.0,russiaukraineconflict,News18 Debate & Interview,2022-10-12T09:48:10.000Z
2544,youtube,When Ukraine Just Invaded Russia... 🇺🇦🇷🇺 #shor...,554.0,17.0,19076.0,russiaukraineconflict,The American Voice,2024-10-04T02:18:08.000Z
